In [1]:
!git clone https://github.com/Jackmin801/StackOverflowMine.git

Cloning into 'StackOverflowMine'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 1), reused 10 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('StackOverflowMine/JavaTop50000.csv')

In [4]:
data.head()

,Id,PostTypeId,AcceptedAnswerId,ParentId,CreationDate,DeletionDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,LastEditorUserId,LastEditorDisplayName,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate,CommunityOwnedDate,ContentLicense,PostId,TagId,Id.1,TagName,Count,ExcerptPostId,WikiPostId
0,11227809,1,11227902.0,NaN,2012-06-27 13:51:36,NaN,24856,1528592,<p>Here is a piece of C++ code that shows some...,87234.0,NaN,224132.0,NaN,2020-07-16 03:35:56,2020-09-15 02:13:25,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,26,2,11113.0,NaN,NaN,CC BY-SA 4.0,11227809,17,17,java,1715860,3624966,3607018
1,6841333,1,6841479.0,NaN,2011-07-27 08:15:58,NaN,6935,667557,"<p>If I run the following program, which parse...",342235.0,NaN,13602076.0,NaN,2020-07-14 19:07:57,2020-07-14 19:07:57,Why is subtracting these two times (in 1927) g...,<java><date><timezone>,10,5,1626.0,NaN,NaN,CC BY-SA 4.0,6841333,17,17,java,1715860,3624966,3607018
2,40480,1,NaN,NaN,2008-09-02 20:14:29,NaN,6724,2024594,<p>I always thought Java uses <strong>pass-by-...,4315.0,NaN,9622275.0,user4642421,2020-08-06 07:24:06,2020-09-16 15:00:21,"Is Java ""pass-by-reference"" or ""pass-by-value""?",<java><methods><parameter-passing><pass-by-ref...,81,1,3053.0,NaN,2012-09-14 18:22:55,CC BY-SA 4.0,40480,17,17,java,1715860,3624966,3607018
3,309424,1,309448.0,NaN,2008-11-21 16:47:40,NaN,4156,2158738,<p>If you have a <code>java.io.InputStream</co...,16616.0,Johnny Maelstrom,63550.0,NaN,2019-01-05 10:28:26,2020-04-05 00:03:47,How do I read / convert an InputStream into a ...,<java><string><io><stream><inputstream>,59,0,1193.0,NaN,NaN,CC BY-SA 4.0,309424,17,17,java,1715860,3624966,3607018
4,271526,1,271874.0,NaN,2008-11-07 08:31:40,NaN,4081,1267072,<p>I use <code>object != null</code> a lot to ...,34856.0,Goran Martinic,11880324.0,cletus,2020-08-17 13:56:41,2020-08-17 13:56:41,How to avoid null checking in Java?,<java><object><nullpointerexception><null>,62,30,1492.0,NaN,2011-10-21 09:02:37,CC BY-SA 4.0,271526,17,17,java,1715860,3624966,3607018


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Id                     50000 non-null  int64  
 1   PostTypeId             50000 non-null  int64  
 2   AcceptedAnswerId       40342 non-null  float64
 3   ParentId               0 non-null      float64
 4   CreationDate           50000 non-null  object 
 5   DeletionDate           0 non-null      float64
 6   Score                  50000 non-null  int64  
 7   ViewCount              50000 non-null  int64  
 8   Body                   50000 non-null  object 
 9   OwnerUserId            48894 non-null  float64
 10  OwnerDisplayName       3204 non-null   object 
 11  LastEditorUserId       37621 non-null  float64
 12  LastEditorDisplayName  1895 non-null   object 
 13  LastEditDate           38097 non-null  object 
 14  LastActivityDate       50000 non-null  object 
 15  Ti

In [8]:
clusters = pd.read_excel('Cluster100B.xlsx', sheet_name=None, index_col=0)
clusters = pd.concat(clusters, ignore_index=True)

In [9]:
del clusters['Title']
del clusters['ViewCount']

In [10]:
data = pd.merge(data, clusters, how='inner', on='Id')

In [11]:
data[['Title','Body','labels']].head()

,Title,Body,labels
0,Why is processing a sorted array faster than p...,<p>Here is a piece of C++ code that shows some...,67
1,Why is subtracting these two times (in 1927) g...,"<p>If I run the following program, which parse...",66
2,"Is Java ""pass-by-reference"" or ""pass-by-value""?",<p>I always thought Java uses <strong>pass-by-...,19
3,How do I read / convert an InputStream into a ...,<p>If you have a <code>java.io.InputStream</co...,87
4,How to avoid null checking in Java?,<p>I use <code>object != null</code> a lot to ...,3


In [12]:
data[['Title','Body','labels']][data['labels']==1]

,Title,Body,labels
166,Can I add jars to maven 2 build classpath with...,<p>Maven2 is driving me crazy during the exper...,1
678,How do I set the proxy to be used by the JVM,"<p>Many times, a Java app needs to connect to ...",1
787,Maven parent pom vs modules pom,<p>There seem to be several ways to structure ...,1
885,What's the best way to validate an XML file ag...,<p>I'm generating some xml files that needs to...,1
967,"In Java, how do I parse XML as a String instea...",<p>I have the following code: </p>\n\n<pre><co...,1
...,...,...,...
49341,What is the correct META-INF directory locatio...,"<p>Should a ""context.xml"" file for deploying a...",1
49413,Convert ivy.xml to pom.xml,"<p>I have a ivy.xml - <a href=""https://gist.gi...",1
49698,What is the difference between Castor XML bind...,<p>What is the difference between Castor XML a...,1
49913,Are there faster XML parsers in Java than Xala...,<p>I haven't found many ways to increase the p...,1


In [13]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
titles = [word_tokenize(str.lower(title)) for title in data.Body.values]

In [15]:
from collections import Counter


# Create a Counter with the lowercase tokens: bow_simple
bow_simple = Counter(titles[0])

# Print the 10 most common tokens
print(bow_simple.most_common(10))

[(';', 65), ('<', 36), ('>', 36), ('&', 30), ('(', 24), (')', 24), ('=', 21), ('lt', 20), ('data', 18), ('c', 14)]


In [16]:
# Import Dictionary
from gensim.corpora.dictionary import Dictionary

# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(titles)

# Use computer_id with the dictionary to print the word
print(dictionary.token2id)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
corpus = [dictionary.doc2bow(title) for title in titles]

# Print the first 10 word ids with their frequency counts from the fifth document
print(corpus[4][:10])

[(0, 2), (4, 8), (5, 2), (6, 2), (11, 1), (13, 1), (17, 2), (18, 5), (20, 5), (21, 1)]


In [18]:
from gensim.models.tfidfmodel import TfidfModel
# Create a new TfidfModel using the corpus: tfidf
tfidf = TfidfModel(corpus)

In [19]:
tfidf[[(0,1),(15,1)]]

[(0, 0.6752269818853636), (15, 0.7376100073439776)]

In [20]:
# Calculate the tfidf weights of doc: tfidf_weights
tfidf_weights = tfidf[corpus[140]]

# Print the first five weights
print(tfidf_weights[:1])

# Sort the weights from highest to lowest: sorted_tfidf_weights
sorted_tfidf_weights = sorted(tfidf_weights, key=lambda w: w[1], reverse=True)

# Print the top 5 weighted words
for term_id, weight in sorted_tfidf_weights[:10]:
    print(dictionary.get(term_id), weight)

[(3, 0.2119144145094103)]
animal 0.5940283225148921
list 0.2528284574172832
animals 0.23655348807848028
& 0.2119144145094103
polymorphism 0.20705419463311528
dog 0.19296825151294686
cat 0.1822236187489883
dosomething 0.17407047055774655
inheritance 0.16892895609057637
generics 0.15842776874223136


In [21]:
group_1 = data[data['labels']==1].index.values

In [22]:
from collections import defaultdict
meow = defaultdict(lambda: 0)
for i in group_1:
  tfidf_weights = tfidf[corpus[i]]
  sorted_tfidf_weights = sorted(tfidf_weights, key=lambda w: w[1], reverse=True)
  for term_id, weight in sorted_tfidf_weights[:5]:
    meow[term_id] += 1


In [23]:
top_10 = sorted(meow.items(), key=lambda item: item[1], reverse=True)[:10]
for i in top_10:
  print(dictionary.get(i[0]),i[1])

xml 68
schema 16
document 11
xsd 9
jaxb 9
sax 9
file 8
import 7
streamresult 7
& 7


In [24]:
from collections import defaultdict
def get_top_10_tfidf(n):
  group = data[data['labels']==n].index.values
  meow = defaultdict(lambda: 0)
  for i in group:
    tfidf_weights = tfidf[corpus[i]]
    sorted_tfidf_weights = sorted(tfidf_weights, key=lambda w: w[1], reverse=True)
    for term_id, weight in sorted_tfidf_weights[:5]:
      meow[term_id] += 1
  top_10 = sorted(meow.items(), key=lambda item: item[1], reverse=True)[:10]
  ret=[]
  for i in top_10:
    #print(dictionary.get(i[0]),i[1])
    ret.append((dictionary.get(i[0]),i[1]/len(group)))
  return ret

In [25]:
highest = [ranks[0] for ranks in [get_top_10_tfidf(i) for i in range(100)]]

In [26]:
for i,h in enumerate(highest):
  print(i,h)

0 ('static', 0.18340611353711792)
1 ('xml', 0.21794871794871795)
2 ('android', 0.2287145242070117)
3 ('%', 0.06405693950177936)
4 ('bigdecimal', 0.9230769230769231)
5 ('spring', 0.20473773265651438)
6 ('at', 0.7410179640718563)
7 ('/kbd', 0.9512195121951219)
8 ('int', 0.23963133640552994)
9 ('enum', 0.7938144329896907)
10 ('@', 0.24660194174757283)
11 ('/li', 0.6957575757575758)
12 ('maven', 0.3687707641196013)
13 ('test', 0.36699029126213595)
14 ('stream', 0.4239766081871345)
15 ('server', 0.21298701298701297)
16 ('date', 0.34057971014492755)
17 ('foo', 0.635593220338983)
18 ('file', 0.3492286115007013)
19 ('br', 0.02860858257477243)
20 ('exception', 0.14496314496314497)
21 ('intellij', 0.49361702127659574)
22 ('thread', 0.5009523809523809)
23 ('node', 0.6814814814814815)
24 ('person', 0.8324607329842932)
25 ('log4j', 0.432258064516129)
26 ('properties', 0.40869565217391307)
27 ('gson', 0.6068965517241379)
28 ('&', 0.4161073825503356)
29 ('b', 0.024139702105803802)
30 ('eclipse', 0.28

In [28]:
df = pd.DataFrame(highest, columns=['Term','Ratio'])
df.sort_values(by='Ratio')[80:]

,Term,Ratio
67,&,0.651016
63,socket,0.663551
40,constructor,0.674757
23,node,0.681481
11,/li,0.695758
96,arraylist,0.711712
32,random,0.728682
69,at,0.740385
6,at,0.741018
44,icon,0.750000
